### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
with open(".env", "w") as f:
    f.write("GROQ_API_KEY=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\n")


In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()


groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:

from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7eef69ae2f10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7eef69af5c90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [5]:
result

AIMessage(content='Here\'s the translation, along with some notes on how to use it:\n\n* **Formal:**\n    * Bonjour (Hello)\n    * Comment allez-vous ? (How are you?) \n\n\n* **Informal:**\n    * Salut (Hello)\n    * Ça va ? (How are you? - This is very casual)\n\n**Important:**\n\n*  In French, it\'s common to use different greetings depending on the level of formality. Use "Bonjour" and "Comment allez-vous ?" with people you don\'t know well or in professional settings. Use "Salut" and "Ça va ?" with friends and family. \n\n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 21, 'total_tokens': 175, 'completion_time': 0.28, 'prompt_time': 0.002126536, 'queue_time': 0.15585670899999998, 'total_time': 0.282126536}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-98ab6fcc-e8a0-4918-814a-b00fcbc5

In [6]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here\'s the translation, along with some notes on how to use it:\n\n* **Formal:**\n    * Bonjour (Hello)\n    * Comment allez-vous ? (How are you?) \n\n\n* **Informal:**\n    * Salut (Hello)\n    * Ça va ? (How are you? - This is very casual)\n\n**Important:**\n\n*  In French, it\'s common to use different greetings depending on the level of formality. Use "Bonjour" and "Comment allez-vous ?" with people you don\'t know well or in professional settings. Use "Salut" and "Ça va ?" with friends and family. \n\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [7]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a couple of ways to say "Hello, How are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is more casual and used with friends and family.)\n\n\n\nLet me know if you\'d like to learn other ways to greet someone in French!\n'

In [8]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [9]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [10]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [11]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [12]:
!pip install fastapi langchain langserve uvicorn pyngrok python-dotenv langchain-groq


In [13]:
from pyngrok import ngrok

# Add your authtoken
ngrok.set_auth_token("2w5vRyVj1ScucIsjXQbtcBN6OTa_78iCrPyB7frMH6k2gEyYH")


In [25]:
!pip install "pydantic<2.0"
!pip install --force-reinstall langserve


  Using cached langserve-0.3.1-py3-none-any.whl.metadata (40 kB)
  Using cached langchain_core-0.3.55-py3-none-any.whl.metadata (5.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.4 MB/s eta 0:00:00
Using cached langserve-0.3.1-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.7 MB/s eta 0:00:00
Using cached langchain_core-0.3.55-py3-none-any.whl (434 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.6/443.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [14]:
ngrok.kill()

In [15]:
import os
import threading
from fastapi import FastAPI
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pyngrok import ngrok
import uvicorn

# Load .env for GROQ_API_KEY
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize Groq model
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

# Define prompt and parser
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])
parser = StrOutputParser()

# Define full chain
chain = prompt_template | model | parser

# Define FastAPI app
app = FastAPI(title="LangChain Server", description="Groq + LangChain API", version="1.0")

# Pydantic model for request body
class InputData(BaseModel):
    language: str
    text: str

@app.post("/chain/invoke")
async def invoke_chain(input: InputData):
    result = chain.invoke({"language": input.language, "text": input.text})
    return {"result": result}

# Start FastAPI in a thread
def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# Give server time to start
import time
time.sleep(2)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"🚀 Server live at: {public_url}/docs")


INFO:     Started server process [10510]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Server live at: NgrokTunnel: "https://1d02-34-34-18-71.ngrok-free.app" -> "http://localhost:8000"/docs


In [17]:
import requests

json_body = {
  "language": "French",
  "text": "Hello"
}

r = requests.post("http://localhost:8000/chain/invoke", json=json_body)
print(r.status_code)
print(r.json())


INFO:     127.0.0.1:51618 - "POST /chain/invoke HTTP/1.1" 200 OK
200
{'result': 'Bonjour \n'}


In [16]:
import requests
import streamlit as st

def get_groq_response(input_text):
    # Send the correct JSON structure expected by FastAPI
    json_body = {
        "language": "French",
        "text": input_text
    }
    try:
        response = requests.post("http://localhost:8000/chain/invoke", json=json_body)
        if response.status_code == 200:
            return response.json().get("result", "No output available")
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Request failed: {e}"

# Streamlit UI
st.title("LLM App using LangChain + Groq")
input_text = st.text_input("Enter English text to convert to French")

if input_text:
    output = get_groq_response(input_text)
    st.success(output)


2025-04-23 08:09:28.333 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 08:09:28.477 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-23 08:09:28.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 08:09:28.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 08:09:28.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 08:09:28.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 08:09:28.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-23 08:09:28.483 Session state does not 

In [18]:
# Start Streamlit and Ngrok
import threading
import time
from pyngrok import ngrok

# Start ngrok tunnel first
public_url = ngrok.connect(8501)
print("🚀 Streamlit app running at:", public_url)

# Run Streamlit in a thread
def run():
    !streamlit run client.py

thread = threading.Thread(target=run)
thread.start()

# Wait a bit to make sure it's running
time.sleep(5)


🚀 Streamlit app running at: NgrokTunnel: "https://b908-34-34-18-71.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.34.18.71:8501



In [19]:
import os

# Create a folder named 'llm_app'
folder_name = "llm_app"
os.makedirs(folder_name, exist_ok=True)



In [21]:
fastapi_code='''
import os
import threading
from fastapi import FastAPI
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from pyngrok import ngrok
import uvicorn

# Load .env for GROQ_API_KEY
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize Groq model
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

# Define prompt and parser
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])
parser = StrOutputParser()

# Define full chain
chain = prompt_template | model | parser

# Define FastAPI app
app = FastAPI(title="LangChain Server", description="Groq + LangChain API", version="1.0")

# Pydantic model for request body
class InputData(BaseModel):
    language: str
    text: str

@app.post("/chain/invoke")
async def invoke_chain(input: InputData):
    result = chain.invoke({"language": input.language, "text": input.text})
    return {"result": result}

# Start FastAPI in a thread
def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

# Give server time to start
import time
time.sleep(2)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"🚀 Server live at: {public_url}/docs")
'''

with open(f"{folder_name}/main_server.py", "w") as f:
    f.write(fastapi_code)

In [22]:
streamlit_code = '''
import requests
import streamlit as st

def get_groq_response(input_text):
    json_body = {
        "language": "French",
        "text": input_text
    }
    try:
        response = requests.post("http://localhost:8000/chain/invoke", json=json_body)
        if response.status_code == 200:
            return response.json().get("result", "No output available")
        else:
            return f"Error: {response.status_code}"
    except Exception as e:
        return f"Request failed: {e}"

# Streamlit UI
st.title("LLM App using LangChain + Groq")
input_text = st.text_input("Enter English text to convert to French")

if input_text:
    output = get_groq_response(input_text)
    st.success(output)
'''

with open(f"{folder_name}/client.py", "w") as f:
    f.write(streamlit_code)


In [32]:
import os
os.getcwd()


'/content'

In [35]:
!ls /content/


client.py  llm_app  requirements.txt  sample_data  serve.py
